### Take-Home 1

#### Task 1.
Integrate information from the “job_url_data” folder into one dictionary. The keys of the dictionary are individual urls scraped from the website, and the values are the earliest date that the corresponding urls were scraped. Save the results into a json file. How many unique job urls have been collected between May 17, 2022 and May 23, 2022?


In [189]:
import json
import os
import pandas as pd
import numpy as np
os.getcwd()
path='/Users/[editted]/Dropbox/work/compsoc/dataset/indeed_scraped_data/job_url_data'
os.chdir(path)
file_list=[i for i in os.listdir(path) if i.endswith('.csv')]
file_list.sort(reverse=True)
print(file_list)

file_dict={}
for item in file_list:
    with open(item, 'r') as f:
        for line in f:
            if line.startswith('http'):
                linec=line.replace('\n', '') # url is a key
                date=item.split('_')[4] # date is a value
                file_dict[linec]=date 
                # {'url': 'date'} 
                # as the file on May, 23 are input to the dictionary first, the date value on May, 17 remains as the value of the dictionary. 
                # Therefore, the duplicated values on the keys are replaced to the earliest date.

with open("/Users/[editted]/Dropbox/work/compsoc/dataset/indeed_scraped_data/job_urls_json.json", "w") as f:
    json.dump(file_dict, f) ## There are 21034 job urls.

['job_urls_for_parsehub_5232022_v2.csv', 'job_urls_for_parsehub_5232022_v1.csv', 'job_urls_for_parsehub_5222022_v2.csv', 'job_urls_for_parsehub_5222022_v1.csv', 'job_urls_for_parsehub_5212022_v2.csv', 'job_urls_for_parsehub_5212022_v1.csv', 'job_urls_for_parsehub_5202022_v2.csv', 'job_urls_for_parsehub_5202022_v1.csv', 'job_urls_for_parsehub_5192022_v2.csv', 'job_urls_for_parsehub_5192022_v1.csv', 'job_urls_for_parsehub_5182022_v1.csv', 'job_urls_for_parsehub_5172022_v5.csv', 'job_urls_for_parsehub_5172022_v4.csv', 'job_urls_for_parsehub_5172022_v3.csv', 'job_urls_for_parsehub_5172022_v2.csv', 'job_urls_for_parsehub_5172022_v1.csv']


In [190]:
# # Diagnostic 1: convert Dictionary to Dataframe and check the data structure.
# file_pd=pd.DataFrame(file_dict.items(), columns=['lnks_link', 'date'])
# file_pd.shape

# # Diagnostic 2: DataFrame List Version to check the dictionary is properly shaped by using lists to arrange the data.
# all_list=[]
# na_list=[]
# for item in file_list:
#     item_list=[]
#     datelist=[]
#     i=0
#     with open(item, 'r') as f:
#         for line in f:
#             if line.startswith('http'):
#                 i+=1
#                 line_n=i
#                 linec=line.replace('\n', '')
#                 date=item.split('_')[4]
#                 item_list.append(linec)
#                 datelist.append(date)
#         na_list.append(line_n)
#         print(item, "n=", line_n)
#         all_list.append({'lnks_link': item_list, 'date': date})
# print(sum(na_list))    # total amount of urls is 60403

# fall=pd.DataFrame()
# for i in range(0, len(all_list)):
#     df_i=pd.DataFrame(all_list[i])
#     fall=fall.append(df_i, ignore_index=True)   
# fall.date=pd.to_datetime(fall.date, format='%m%d%Y')
# fall.sort_values('date', inplace=True)
# fall=fall.reset_index(drop=True)
# fall['dup']=fall.duplicated(subset='lnks_link', keep='first')
# fall=fall[fall.dup==False].drop(columns='dup',axis=1).reset_index(drop=True)
# fall.to_csv('/Users/gohtk/Dropbox/work/compsoc/dataset/indeed_scraped_data/job_urls_cleaned.csv', index=False)
# fall # after cleaning the duplicates, n=21034

#### Task 2. 
Clean and integrate information from the “job_info_data” folder into one data frame. Files from this subfolder might have two different formats. Some of them are csv files, while others are json files. The columns might also be named differently. Find ways to read each of the files into pandas, drop records with missing job titles and/or missing job descriptions, and combine them into one dataframe. Lastly, drop records with duplicate job urls, and then save them into a separate csv file. How many unique jobs are there in the cleaned dataframe?


In [191]:
path2='/Users/[editted]/Dropbox/work/compsoc/dataset/indeed_scraped_data/job_info_data'
os.chdir(path2)
file_list=[i for i in os.listdir(path2) if i.endswith(('.csv', '.json'))]
file_list.sort(reverse=True)
print(file_list)

info_pd=pd.DataFrame()

for item in file_list:
    if item.endswith('.csv'):
        c=pd.read_csv(item)
        info_pd=pd.concat([info_pd, c])
    elif item.endswith('.json'):
        j = pd.read_json(item)
        j['lnks_link']=None
        for i in range(0, len(j)):
            for v, k in j.lnks[i].items():
                j.lnks_link[i]=k
        j.lnks_link=j.lnks_link.replace(r'', np.NaN) 
        j=j.dropna().drop(['lnks'], axis=1) 
        print(item, j.shape, j.columns)
        info_pd=pd.concat([info_pd, j])
info_pd=info_pd.dropna(subset=['lnks_job_title', 'lnks_job_description']).drop_duplicates(subset='lnks_link').reset_index(drop=True)
print(info_pd.head())
info_pd.to_csv('/Users/[editted]/Dropbox/work/compsoc/dataset/indeed_scraped_data/job_info_cleaned.csv', index=False)

# n=14200

['parsehub_5232022_v2.csv', 'parsehub_5232022_v1.csv', 'parsehub_5222022_v6.csv', 'parsehub_5222022_v5.csv', 'parsehub_5222022_v4.csv', 'parsehub_5222022_v3.csv', 'parsehub_5222022_v2.csv', 'parsehub_5222022_v1.csv', 'parsehub_5212022_v2.csv', 'parsehub_5212022_v1.csv', 'parsehub_5202022_v2.json', 'parsehub_5202022_v1.csv', 'parsehub_5192022_v2.csv', 'parsehub_5192022_v1.csv', 'parsehub_5182022_v2.json', 'parsehub_5182022_v1.csv', 'parsehub_5172022_v2.csv', 'parsehub_5172022_v1.csv']
parsehub_5202022_v2.json (4857, 1) Index(['lnks_link'], dtype='object')
parsehub_5182022_v2.json (2833, 1) Index(['lnks_link'], dtype='object')
                                           lnks_link  \
0  https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...   
1  https://www.indeed.com/rc/clk?jk=2e84a8f5beb07...   
2  https://www.indeed.com/rc/clk?jk=30d7f8247c497...   
3  https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...   
4  https://www.indeed.com/rc/clk?jk=ee370349db8d8...   

                             l

#### Task 3. 
Merge between “job_url_data” and “job_info_data”. What is the percentage of jobs that can be matched between these two data sources? How are the missing data (unmatched job urls) distributed by date? What about matched job urls? How many complete job listings were we able to collect each day? How would you interpret this result with respect to data quality? Does it mean that our data collection strategy is flawed and thus introduces non-random sampling biases?


In [192]:
os.chdir('/Users/[editted]/Dropbox/work/compsoc/dataset/indeed_scraped_data')

with open('job_urls_json.json', 'r') as j:
    juj_json=json.load(j)
juj=pd.DataFrame.from_dict(juj_json, orient='index').reset_index().rename({'index': 'lnks_link', 0: 'date'}, axis=1)
jic=pd.read_csv('job_info_cleaned.csv')
job_urls_merge = pd.merge(juj, jic, how='left', on='lnks_link').sort_values('date').reset_index(drop=True)

In [196]:
# What is the percentage of jobs that can be matched between these two data sources?
print(job_urls_merge.lnks_job_title.isna().value_counts())
tf=job_urls_merge.lnks_job_title.isna().value_counts().to_list()
matched=tf[0]
unmatched=tf[1]
print(matched/(matched+unmatched)*100)
# the rate of matched jobs to the merged data of "job_url_data" and "job_info_data" is 66.8%

False    14050
True      6984
Name: lnks_job_title, dtype: int64
66.79661500427879


In [199]:
# How are the missing data (unmatched job urls) distributed by date? What about matched job urls?
datelist=pd.DataFrame(job_urls_merge.date.drop_duplicates()).reset_index().date
job_d=pd.DataFrame()

for day in datelist:
    job_date=job_urls_merge.loc[job_urls_merge.date==day]
    job_dateday=pd.DataFrame(job_date.lnks_job_title.isna().value_counts()).reset_index().rename({'index': 'result', 'lnks_job_title': 'job_counts'}, axis=1)
    job_dateday['date']=day
    job_d=pd.concat([job_d,job_dateday])

job_d.result=job_d.result.map({True: 'True', False: 'False'}).replace('True', 'Not Matched').replace('False', 'Matched')
job_d=job_d.sort_values(by=['result']).reset_index(drop=True) 
job_d

,result,job_counts,date
0,Matched,2610,5172022
1,Matched,672,5182022
2,Matched,3560,5192022
3,Matched,4445,5202022
4,Matched,2098,5212022
5,Matched,665,5222022
6,Not Matched,2607,5172022
7,Not Matched,630,5182022
8,Not Matched,193,5192022
9,Not Matched,20,5202022


In [200]:
# How many complete job listings were we able to collect each day? How would you interpret this result with respect to data quality? 
# Does it mean that our data collection strategy is flawed and thus introduces non-random sampling biases?

total_date=job_urls_merge.groupby('date').agg({'lnks_link': 'count'}).rename({'lnks_link':'total job listing'}, axis=1).reset_index()
c_list=job_urls_merge.columns.to_list()
job_complete=job_urls_merge.dropna(subset=c_list).groupby('date').agg({'lnks_link': 'count'}).rename({'lnks_link':'complete job listing'}, axis=1).reset_index()
job_dm=job_d[job_d.result=='Matched'].rename({'job_counts': 'matched_job_counts'}, axis=1)
job_dnm=job_d[job_d.result=='Not Matched'].rename({'job_counts': 'unmatched_job_counts'}, axis=1)

for item in job_complete, job_dm, job_dnm:
    total_date=total_date.merge(item, how='left', on='date')
total_date=total_date.drop(columns=['result_x', 'result_y']).replace(np.NaN, 0)
total_date['data quality_complete']=total_date['complete job listing']/total_date['total job listing']*100
total_date['data quality_match']=total_date['matched_job_counts']/total_date['total job listing']*100    
total_date

,date,total job listing,complete job listing,matched_job_counts,unmatched_job_counts,data quality_complete,data quality_match
0,5172022,5217,2208.0,2610.0,2607,42.323174,50.028752
1,5182022,1302,634.0,672.0,630,48.694316,51.612903
2,5192022,3753,3174.0,3560.0,193,84.572342,94.857447
3,5202022,4465,4130.0,4445.0,20,92.497200,99.552072
4,5212022,2139,1732.0,2098.0,41,80.972417,98.083216
5,5222022,777,539.0,665.0,112,69.369369,85.585586
6,5232022,3381,0.0,0.0,3381,0.000000,0.000000


The 'complete job listing' column in the data frame shows the number of completed job lists by each day.
In terms of data quality, I use two measures of data quality: the rate of the number of complete job lists to the total number of job lists and the rate of the number of matched job lists to the total number of job lists.
The former refers to the percentage of job lists that have full information in the job_info_data. The results by date are in the column, 'data quality_complete'. The job urls collected on May 20 shows the highest percentage of completed information while the job urls on May 17 and May 18 are less than 50 percent of completed job information. 
The latter indicates the percentage of job lists that have the information of links matched with job title. The results by date are in the 'data quality_match' column. 
The problem in the quality of this data is that there is no matched job urls with job information in May 23, 2022. I think the "job_info_data" have to be checked again whether it is collected properly. 
Considering the fact that the amount of job listings on May 23 is more than 3,000, it might cause the selection bias in the sample related to the seasonal or periodical patterns of job posting. 

#### Bonus Task 1.
Using the merged dataset from Task 3, extract state information for each individual job based on the “company_location” column. Aggregate them by state, and create a state-level choropleth map to visualize the spatial distribution of compliance jobs. The map should be colored based on the total number of compliance jobs in each state. The boundaries of US states in geojson can be found here. Interpret the results. [You will get three extra points if you are able to finish this bonus task.]